In [21]:
# dependencies
import pandas as pd
import numpy as np
import csv

In [22]:
csv_schools_path = "Resources/schools_complete.csv"
csv_students_path ="Resources/students_complete.csv"

schools_complete_df = pd.read_csv(csv_schools_path)
students_complete_df = pd.read_csv(csv_students_path)
schools_complete_df.shape

(15, 5)

In [23]:
#clean up school dataframe:
schools_complete_df=schools_complete_df.drop('School ID',axis=1)
schools_clean_df = schools_complete_df.sort_values(by=['school_name'])
schools_clean_df = schools_clean_df.set_index('school_name')

#Rename column header for consistency
students_clean_df = students_complete_df.rename(columns={'Student ID': 'student_id'})

#Merge both csv files
pycityschools_df = pd.merge(schools_clean_df,students_clean_df,on="school_name")
pycityschools_df.head()

,school_name,type,size,budget,student_id,student_name,gender,grade,reading_score,math_score
0,Bailey High School,District,4976,3124928,17871,Blake Martin,M,9th,75,59
1,Bailey High School,District,4976,3124928,17872,Kathryn Kane,F,12th,84,58
2,Bailey High School,District,4976,3124928,17873,Richard Haas,M,11th,79,86
3,Bailey High School,District,4976,3124928,17874,Frank Marsh,M,9th,71,89
4,Bailey High School,District,4976,3124928,17875,Charles Goodman Jr.,M,9th,90,61


In [24]:
#Calculate number of schools:
school_count = len(pycityschools_df.school_name.unique())

#Calculate number of students 
    #used student_id to ensure no duplication of names
student_count = len(pycityschools_df.student_id.unique())


#Calculate total budget using original school csv file:
#school_budget=schools_clean_df.budget.unique()
total_budget = schools_clean_df.budget.sum(axis=0)

#Caculate ave scores for math and reading
ave_math_score = round(pycityschools_df.math_score.mean(),2)
ave_reading_score = round(pycityschools_df.reading_score.mean(),2)

#Count& % of students passing math:
math_pass_list= pycityschools_df.loc[pycityschools_df['math_score']>=70]['math_score'].count()
percentage_pass_math = round((math_pass_list/student_count*100),2)

#Count & % of students passing reading:
reading_pass_list = pycityschools_df.loc[pycityschools_df['reading_score']>=70]['reading_score'].count()
percentage_pass_reading = round((reading_pass_list/student_count*100),2)

#Count and % of overll passing - must pass both reading & math:
overall_pass_list = pycityschools_df[(pycityschools_df['math_score']>=70) & (pycityschools_df['reading_score']>=70)]['student_id'].count()
percentage_pass_overall = round((overall_pass_list/student_count*100),2)

#Formatting dictionary:
format_dict={'Total Budget': '${0:,.0f}',
             'Total School Budget': '${0:,.0f}',
             'Per Student Budget': '${0:,.0f}', 
             'Ave Math Score': '{:1}', 
             'Ave Reading Score': '{:1}' , 
             '% Passing Math': '{:1}%', 
             '% Passing Reading': '{:1}%', 
             '% Overall Passing': '{:1}%'}

In [25]:
#Key metrics table - district level
district_data =[{'Total Schools':school_count,
                 'Total Students': student_count,
                 'Total Budget': total_budget, 
                 'Ave Math Score':ave_math_score,
                 'Ave Reading Score':ave_reading_score,
                 '% Passing Math':percentage_pass_math,
                 '% Passing Reading':percentage_pass_reading,
                 '% Overall Passing': percentage_pass_overall}]
data_district_summary_df=pd.DataFrame(district_data)

data_district_summary_df=data_district_summary_df.style.format(format_dict)

data_district_summary_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


In [26]:
#School Summary Calculations

schools = pycityschools_df.groupby(['school_name'])['school_name'].first()
school_type = pycityschools_df.groupby(['school_name'])['type'].first()
schools_student_count = pycityschools_df.groupby(['school_name'])['student_id'].count()
school_budget = pycityschools_df.groupby(['school_name'])['budget'].first()
student_budget = school_budget/schools_student_count
school_ave_math = round(pycityschools_df.groupby(['school_name'])['math_score'].mean(),2)
school_ave_reading = round(pycityschools_df.groupby(['school_name'])['reading_score'].mean(),2)
school_math_perc=round((pycityschools_df[pycityschools_df['math_score']>=70].groupby('school_name')['student_id'].count())/(schools_student_count)*100,2)
school_reading_perc=round((pycityschools_df[pycityschools_df['reading_score']>=70].groupby('school_name')['student_id'].count())/(schools_student_count)*100,2)
overall_pass_count=pycityschools_df[(pycityschools_df['math_score']>=70) & (pycityschools_df['reading_score']>=70)].groupby('school_name')['student_id'].count()
overall_pass_perc=round((overall_pass_count/schools_student_count)*100,2)

In [30]:
#School Summary Printout
school_summary_df = pd.concat([schools,school_type,schools_student_count,school_budget,student_budget,school_ave_math,
             school_ave_reading,school_math_perc,school_reading_perc,overall_pass_perc],axis=1)
school_summary_df =school_summary_df.set_index('school_name')
school_summary_df.columns=['School Type', 'Total Students', 'Total Budget','Per Student Budget', 'Ave Math Score', 
                          'Ave Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']
school_summary_df.rename_axis('School Name', inplace = True)
school_summary_format_df = school_summary_df.style.format(format_dict)
school_summary_format_df

,School Type,Total Students,Total Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.2%
Ford High School,District,2739,"$1,763,916",$644,77.1,80.75,68.31%,79.3%,54.29%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.6%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.8,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


In [31]:
#Top 5 Performing Schools:
top_5_schools_df = school_summary_df.sort_values(by='% Overall Passing',ascending=False).head(5)
top_5_schools_df = top_5_schools_df.style.format(format_dict)
top_5_schools_df

,School Type,Total Students,Total Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.6%
Wilson High School,Charter,2283,"$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


In [32]:
#Bottom 5 Performing Schools:
bottom_5_schools_df=school_summary_df.sort_values(by='% Overall Passing',ascending=True).head(5)
bottom_5_schools_df=bottom_5_schools_df.style.format(format_dict)
bottom_5_schools_df

,School Type,Total Students,Total Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.2%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


In [33]:
#Re-index dataframe and group by grade level:
scores_by_grade_df = pycityschools_df.set_index('grade').groupby(['grade'])

In [34]:
#Math scores by Grade Level:
math_scores_grade = round(scores_by_grade_df['math_score'].mean(),2)
math_scores_grade_df=pd.DataFrame(math_scores_grade)
math_scores_grade_df.columns=['Ave Math Score']
math_scores_grade_df

,Ave Math Score
grade,
10th,78.94
11th,79.08
12th,78.99
9th,78.94


In [35]:
#Reading scores by Grade Level:
reading_scores_grade = round(scores_by_grade_df['reading_score'].mean(),2)
reading_scores_grade_df=pd.DataFrame(reading_scores_grade)
reading_scores_grade_df.columns=['Ave Reading Score']
reading_scores_grade_df

,Ave Reading Score
grade,
10th,81.87
11th,81.89
12th,81.82
9th,81.91


In [43]:
#Binning school budgets for: Scores by School Spending:
school_spending_df=pycityschools_df
spending_bins=[0,999999,1249999,1999999,4000000]
spending_names=["<$1,000,000", " <$1,250,000","<$2,000,000",">$2,000,000"]
school_spending_df['Budget Limits']=pd.cut(school_spending_df['budget'],bins=spending_bins,labels=spending_names)
school_spending_df

spending_school_df = school_spending_df.set_index('Budget Limits').groupby(['Budget Limits'])


In [52]:
ave_math_by_spending=round(spending_school_df['math_score'].mean(),2)
ave_reading_by_spending = round(spending_school_df['reading_score'].mean(),2)
spending_student_count=spending_school_df['student_id'].count()
spending_math_perc=round((school_spending_df[school_spending_df['math_score']>=70].groupby('Budget Limits')['student_id'].count())/(spending_student_count)*100,2)
spending_reading_perc=round((school_spending_df[school_spending_df['reading_score']>=70].groupby('Budget Limits')['student_id'].count())/(spending_student_count)*100,2)
spending_pass_count = school_spending_df[(school_spending_df['math_score']>=70) & (school_spending_df['reading_score']>=70)].groupby('Budget Limits')['student_id'].count()
spending_pass_perc=round((spending_pass_count/spending_student_count)*100,2)

spending_dict ={'Ave Math Score': ave_math_by_spending,
                'Ave Reading Score': ave_reading_by_spending,
                '% Passing Math': spending_math_perc,
                '% Passing Reading': spending_reading_perc,
                '% Overall Passing': spending_pass_perc}
spending_summary_df = pd.DataFrame(spending_dict)
#spending_summary_df.columns=['Budget Limits','Ave Math Score','Ave Reading Score','% Passing Math','% Passing Reading','% Overall Passing']
#spending_summary_df =spending_summary_df.set_index('Budget Limits')
spending_summary_df

,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Budget Limits,,,,,
"<$1,000,000",83.58,83.89,93.66,96.60,90.37
"<$1,250,000",83.38,83.88,93.66,96.70,90.63
"<$2,000,000",78.16,81.65,72.34,83.84,61.40
">$2,000,000",77.07,80.93,66.47,81.11,53.72


In [ ]:
#School Summary Printout
school_summary_df = pd.concat([schools,school_type,schools_student_count,school_budget,student_budget,school_ave_math,
             school_ave_reading,school_math_perc,school_reading_perc,overall_pass_perc],axis=1)
school_summary_df =school_summary_df.set_index('school_name')
school_summary_df.columns=['School Type', 'Total Students', 'Total Budget','Per Student Budget', 'Ave Math Score', 
                          'Ave Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']
school_summary_df.rename_axis('School Name', inplace = True)
school_summary_format_df = school_summary_df.style.format(format_dict)
school_summary_format_df

In [42]:
#Scores by School Spending

#Calcuation for ave math score by spending bins & create dataframe
ave_math_by_spending=round(spending_school_df['math_score'].mean(),2)
#ave_math_by_spending_df=pd.DataFrame(ave_math_by_spending)

#Calcuation for ave reading score by spending bins & create dataframe
ave_reading_by_spending = round(spending_school_df['reading_score'].mean(),2)
#ave_reading_by_spending_df = pd.DataFrame(ave_reading_by_spending)

#Count of students by spending bins:
spending_student_count=spending_school_df['student_id'].count()

#Calculations of Math passing percentages by spending bins:
spending_math_perc=round((school_spending_df[school_spending_df['math_score']>=70].groupby('Budget Limits')['student_id'].count())/(spending_student_count)*100,2)
#spending_math_perc_df=pd.DataFrame(spending_math_perc)

#Calculations of Reading passing percentages by spending bins:
spending_reading_perc=round((school_spending_df[school_spending_df['reading_score']>=70].groupby('Budget Limits')['student_id'].count())/(spending_student_count)*100,2)
#spending_reading_perc_df=pd.DataFrame(spending_math_perc)
#spending_reading_perc_df

#Calculations of Overall passing percentages by spending bins:
spending_pass_count = school_spending_df[(school_spending_df['math_score']>=70) & (school_spending_df['reading_score']>=70)].groupby('Budget Limits')['student_id'].count()
spending_pass_perc=round((spending_pass_count/spending_student_count)*100,2)
#pending_pass_perc_df=pd.DataFrame(spending_pass_perc)

spending_summary_df = pd.concat([spending_bins,schools,ave_math_by_spending,ave_reading_by_spending,spending_math_perc,spending_reading_perc,spending_pass_perc], axis=1)
spending_summary_df =spending_summary_df.set_index('Budget Limits')
spending_summary_df.columns=['Budget Limits','Ave Math Score','Ave Reading Score','% Passing Math','% Passing Reading','% Overall Passing']

#Merge ave Math & Reading DataFrames
#spending_summary_df=pd.merge(ave_math_by_spending_df,ave_reading_by_spending_df,how='left',on='Budget Limits')
#spending_summary_df=pd.merge(spending_summary_df, spending_math_perc_df,how='left',on='Budget Limits')
#spending_summary_df=pd.merge(spending_summary_df,spending_reading_perc_df,how='left',on='Budget Limits')
#spending_summary_df=pd.merge(spending_summary_df,spending_pass_perc_df,how='left',on='Budget Limits')
#spending_summary_df.columns=['Ave Math Score', 'Ave Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']

#format_dict={'Ave Math Score': '{:1}', 'Ave Reading Score': '{:1}' , '% Passing Math': '{:1}%', '% Passing Reading': '{:1}%', '% Overall Passing': '{:1}%'}
#spending_summary_df=spending_summary_df.style.format(format_dict)

#Display table
spending_summary_df

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [14]:
#Scores by School Size

#copy original dataframe
size_school_df=pycityschools_df

#Binning school sizes for Scores by School Sizes:
size_bins=[0, 1800, 3000, 5000]
size_names=['Small', 'Medium', 'Large']
size_school_df['School Size']=pd.cut(size_school_df['size'], bins=size_bins, labels=size_names)

#set index & groupby
school_size_df = size_school_df.set_index('School Size').groupby(['School Size'])

#Calcuation for ave math score by school size & create dataframe
ave_math_by_size=round(school_size_df['math_score'].mean(),2)
ave_math_by_size_df=pd.DataFrame(ave_math_by_size)

#Calcuation for ave reading score by school size & create dataframe
ave_reading_by_size = round(school_size_df['reading_score'].mean(),2)
ave_reading_by_size_df = pd.DataFrame(ave_reading_by_size)

#Count of students by size bins:
size_student_count=school_size_df['student_id'].count()

#Calculations of Math passing percentages by school size:
size_math_perc=round((size_school_df[size_school_df['math_score']>=70].groupby('School Size')['student_id'].count())/(size_student_count)*100,2)
size_math_perc_df=pd.DataFrame(size_math_perc)

#Calculations of Reading passing percentages by school size:

size_reading_perc=round((size_school_df[size_school_df['reading_score']>=70].groupby('School Size')['student_id'].count())/(size_student_count)*100,2)
size_reading_perc_df=pd.DataFrame(size_reading_perc)

#Calculations of Overall passing percentages by school size
size_pass_count = size_school_df[(size_school_df['math_score']>=70) & (size_school_df['reading_score']>=70)].groupby('School Size')['student_id'].count()
size_pass_perc=round((size_pass_count/size_student_count)*100,2)
size_pass_perc_df=pd.DataFrame(size_pass_perc)

#Merge ave Math & Reading DataFrames
size_summary_df=pd.merge(ave_math_by_size_df,ave_reading_by_size_df,how='left',on='School Size')
size_summary_df=pd.merge(size_summary_df, size_math_perc_df,how='left',on='School Size')
size_summary_df=pd.merge(size_summary_df, size_reading_perc_df,how='left',on='School Size')
size_summary_df=pd.merge(size_summary_df, size_pass_perc_df,how='left',on='School Size')
size_summary_df.columns=['Ave Math Score', 'Ave Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']

#format columns:
#format_dict={'Ave Math Score': '{:1}', 'Ave Reading Score': '{:1}' , '% Passing Math': '{:1}%', '% Passing Reading': '{:1}%', '% Overall Passing': '{:1}%'}
size_summary_df=size_summary_df.style.format(format_dict)
size_summary_df


,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small,83.52,83.86,93.56%,96.59%,90.38%
Medium,78.88,81.99,75.51%,85.77%,65.76%
Large,77.07,80.93,66.47%,81.11%,53.72%


In [17]:
# Scores by School Type

#copy original dataframe
type_school_df=pycityschools_df

#Groupby type of school
school_type_df = type_school_df.set_index('type').groupby(['type'])

#Calcuation for ave math score by school type & create dataframe
ave_math_by_type=round(school_type_df['math_score'].mean(),2)
ave_math_by_type_df=pd.DataFrame(ave_math_by_type)

#Calcuation for ave reading score by school type & create dataframe
ave_reading_by_type = round(school_type_df['reading_score'].mean(),2)
ave_reading_by_type_df = pd.DataFrame(ave_reading_by_type)

#Count of students by type:
type_student_count=school_type_df['student_id'].count()

#Calculations of Math passing percentages by school type:
type_math_perc=round((pycityschools_df[pycityschools_df['math_score']>=70].groupby('type')['student_id'].count())/(type_student_count)*100,2)
type_math_perc_df=pd.DataFrame(type_math_perc)

#Calculations of Reading passing percentages by school type:
type_reading_perc=round((pycityschools_df[pycityschools_df['reading_score']>=70].groupby('type')['student_id'].count())/(type_student_count)*100,2)
type_reading_perc_df=pd.DataFrame(type_reading_perc)
type_reading_perc_df

#Calculations of Overall passing percentages by school type:
type_pass_count = pycityschools_df[(pycityschools_df['math_score']>=70) & (pycityschools_df['reading_score']>=70)].groupby('type')['student_id'].count()
type_pass_perc=round((type_pass_count/type_student_count)*100,2)
type_pass_perc_df=pd.DataFrame(type_pass_perc)

#Merge ave Math & Reading DataFrames
type_summary_df=pd.merge(ave_math_by_type_df,ave_reading_by_type_df,how='left',on='type')
type_summary_df=pd.merge(type_summary_df,type_math_perc_df,how='left',on='type')
type_summary_df=pd.merge(type_summary_df,type_reading_perc_df,how='left',on='type')
type_summary_df=pd.merge(type_summary_df,type_pass_perc_df,how='left',on='type')
type_summary_df.columns=['Ave Math Score', 'Ave Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']

#Formatting Columns:
#format_dict={'Ave Math Score': '{:1}', 'Ave Reading Score': '{:1}' , '% Passing Math': '{:1}%', '% Passing Reading': '{:1}%', '% Overall Passing': '{:1}%'}
type_summary_df=type_summary_df.style.format(format_dict)
type_summary_df

,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.41,83.9,93.7%,96.65%,90.56%
District,76.99,80.96,66.52%,80.91%,53.7%
